In [22]:
import requests

API_TOKEN = "enter your api token"
API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-large"
headers = {"Authorization": f"Bearer {API_TOKEN}"}



# Method 1 : using API call

In [23]:
def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()



In [16]:
output = query(".\AIDER\collapsed_building\collapsed_building_image0001.jpg")

In [17]:
output

[{'generated_text': 'arafed roof of a building is seen in this aerial photo'}]

# Method 2 : using model load

In [18]:
from PIL import Image
import requests
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

image_path = ".\AIDER\collapsed_building\collapsed_building_image0001.jpg"
image = Image.open(image_path)
#text = "A picture of"

#inputs = processor(images=image, text=text, return_tensors="pt")

#outputs = model(**inputs)

In [19]:
# prepare image for the model
inputs = processor(images=image, return_tensors="pt")
pixel_values = inputs.pixel_values

generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)

a large building that has been damaged by the tornado


# run for all images of AIDER

In [24]:
import os 
import pandas as pd

In [31]:
def generate_caption_for_image(input_image_path):
    image = Image.open(input_image_path)
    # prepare image for the model
    inputs = processor(images=image, return_tensors="pt")
    pixel_values = inputs.pixel_values

    generated_ids = model.generate(pixel_values=pixel_values, max_length=100)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    #print(generated_caption)
    return generated_caption
   

In [20]:
folder_list = ['collapsed_building','fire', 'flooded_areas','traffic_incident']

In [26]:
base_folder_path = r".\AIDER"

In [41]:
# Define a dictionary to map folder names to label classes
label_class = {'collapsed_building': 0, 'fire': 1, 'flooded_areas': 2, 'traffic_incident': 3}

# Initialize your data list
data = []

In [42]:

# Iterate through each folder
for folder_name in folder_list:
    folder_path = os.path.join(base_folder_path, folder_name)
    label = label_class[folder_name]  # Get the label for the folder
    
    # Iterate through files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Construct the full image path
            image_path = os.path.join(folder_path, filename)
            
            # Generate captions for the image (replace with your caption generation code)
            caption = generate_caption_for_image(image_path)  # Replace with your code
            
            # Create a data entry dictionary
            entry = {
                'File Name': filename,
                'Caption': caption,
                'image_file_path': image_path,
                'label_class': label
            }
            
            # Append the entry to the data list
            data.append(entry)



In [43]:
len(data)

2043

In [44]:
caption_image_df = pd.DataFrame(data)


In [45]:
caption_image_df.head(5)

,File Name,Caption,image_file_path,label_class
0,collapsed_building_image0001.jpg,a large building that has been damaged by the ...,.\AIDER\collapsed_building\collapsed_building_...,0
1,collapsed_building_image0002.jpg,a house fire in the area of a residential area,.\AIDER\collapsed_building\collapsed_building_...,0
2,collapsed_building_image0003.jpg,a house is shown in this aerial photo,.\AIDER\collapsed_building\collapsed_building_...,0
3,collapsed_building_image0004.jpg,a house is seen in the aftermath of a tornado ...,.\AIDER\collapsed_building\collapsed_building_...,0
4,collapsed_building_image0005.jpg,the scene of the explosion at the mall in the ...,.\AIDER\collapsed_building\collapsed_building_...,0


In [46]:
caption_image_df.to_csv('captions_AIDER_class_generate_2.csv', index=False)


In [38]:
data [:5]

[{'File Name': 'collapsed_building_image0001.jpg',
  'Caption': 'a large building that has been damaged by the tornado'},
 {'File Name': 'collapsed_building_image0002.jpg',
  'Caption': 'a house fire in the area of a residential area'},
 {'File Name': 'collapsed_building_image0003.jpg',
  'Caption': 'a house is shown in this aerial photo'},
 {'File Name': 'collapsed_building_image0004.jpg',
  'Caption': 'a house is seen in the aftermath of a tornado in moore, texas'},
 {'File Name': 'collapsed_building_image0005.jpg',
  'Caption': 'the scene of the explosion at the mall in the bronx'}]